In [ ]:
"""
    ____  __________  _________    ____  ________  _______ 
   / __ \/ ____/ __ \/  _/ ___/   / __ \/ ____/  |/  / __ \
  / /_/ / __/ / / / // / \__ \   / / / / __/ / /|_/ / / / /
 / _, _/ /___/ /_/ // / ___/ /  / /_/ / /___/ /  / / /_/ / 
/_/ |_/_____/_____/___//____/  /_____/_____/_/  /_/\____/ 
"""

In [242]:
# Import and instanciate redis object

import redis
r = redis.Redis()  # No params defaults to Host: localhost and Port: 6379

# Remove everithing that is in the databse so we can start from scratch
r.flushall()

True

## STRING / INTEGER

In [248]:
# CREATE VALUES
print("----- CREATE -----")
r.set("name", "Bern")
r.set("population", "1030000")
r.set("greeting", "Hello world")
r.setnx("population", "100") # -> This will not be set due to existing key `population`
print('Set these strings:', r.get("name"), r.get("population"), r.get("greeting"))

# READ VALUES FROM DB
print("\n----- READ -----")
print(f'Get by key: {r.get("name")}')
print(f'Get mutiple by key: {r.mget("name", "population")}')
print(f'Lenght of the key `Bern` is: {r.strlen("name")}')

# UPDATE VALUES IN DB
print("\n----- UPDATE -----")
r.setrange("greeting", 6, "REDIS")
print(f'Greeting changed from `Hello world` to: {r.get("greeting")}')
print(f'Increase population by 1: {r.incr("population")}')
print(f'Increase population by 99: {r.incrby("population", 99)}')

# DELETE VALUES FROM DB
print("\n----- DELETE -----")
r.delete("greeting")
print(f'Deleted key `greeting` - value is: {r.get("greeting")}')


----- CREATE -----
Set these strings: b'Bern' b'1030000' b'Hello world'

----- READ -----
Get by key: b'Bern'
Get mutiple by key: [b'Bern', b'1030000']
Lenght of the key `Bern` is: 4

----- UPDATE -----
Greeting changed from `Hello world` to: b'Hello REDIS'
Increase population by 1: 1030001
Increase population by 99: 1030100

----- DELETE -----
Deleted key `greeting` - value is: None


## LIST

In [204]:
# For demo puroposes clear the list every the time we run this cell
r.delete("buildings")

# CREATE VALUES
print("----- CREATE -----")
r.lpush("buildings", *["Bundehaus", "Kornhaus"])
print('Add item to list', r.lrange("buildings", 0, 10))

r.lpush("buildings", "Baldachin")
print('Push to the beginning:', r.lrange("buildings", 0, 10))

r.rpush("buildings", "Zytglogge")
print('Push to the end:', r.lrange("buildings", 0, 10))

r.lset("buildings", 2, "Rathaus")
print('Set item by index:', r.lrange("buildings", 0, 10))

# READ VALUES FROM DB
print("\n----- READ -----")
print(f'Get single list item: {r.lindex("buildings", 2)}')
print(f'Get range of items:. {r.lrange("buildings", 0, 1)}')

# UPDATE VALUES IN DB
print("\n----- UPDATE -----")
print('Pop from the start:', r.lpop("buildings"))
print('Pop from the end:', r.rpop("buildings"))
r.rpush("buildings", *["Marktgasse", "Marktgasse"])

# DELETE VALUES FROM DB
print("\n----- DELETE -----")
print('Remove item by index:', r.lrange("buildings", 0, 10), r.lrem("buildings", -2, "Marktgasse"), r.lrange("buildings", 0, 10))



----- CREATE -----
Add item to list [b'Kornhaus', b'Bundehaus']
Push to the beginning: [b'Baldachin', b'Kornhaus', b'Bundehaus']
Push to the end: [b'Baldachin', b'Kornhaus', b'Bundehaus', b'Zytglogge']
Set item by index: [b'Baldachin', b'Kornhaus', b'Rathaus', b'Zytglogge']

----- READ -----
Get single list item: b'Rathaus'
Get range of items:. [b'Baldachin', b'Kornhaus']

----- UPDATE -----
Pop from the start: b'Baldachin'
Pop from the end: b'Zytglogge'

----- DELETE -----
Remove item by index: [b'Kornhaus', b'Rathaus', b'Marktgasse', b'Marktgasse'] 2 [b'Kornhaus', b'Rathaus']


## SET

In [243]:
# For demo puroposes clear the sets every the time we run this cell
r.delete("districts", "districts2")

# CREATE VALUES
print("----- CREATE -----")
r.sadd("districts", *["Breitsch", "Lorraine", "Matte", "Obstberg"])
r.sadd("districts2", *["Lorraine", "Ostring"])
print('Create 2 set with multiple values:', r.smembers("districts"), r.smembers("districts2"))

# READ VALUES FROM DB
print("\n----- READ -----")
print('Get reandom entry from the set:', r.srandmember("districts"))
print('Intersection between the two sets:', r.sinter("districts", "districts2"))
print('Difference between the two sets:', r.sdiff("districts", "districts2"))
print('Union of the two sets:', r.sunion("districts", "districts2"))

# UPDATE VALUES IN DB
print("\n----- UPDATE -----")
r.smove("districts", "districts2", "Matte")
print('Move `Matte` from first to second set:', r.smembers("districts"), r.smembers("districts2"))

# DELETE VALUES FROM DB
print("\n----- DELETE -----")
r.srem("districts", "Breitsch")
print('Remove `Breitsch` from the first set:', r.smembers("districts"))


----- CREATE -----
Create 2 set with multiple values: {b'Matte', b'Obstberg', b'Breitsch', b'Lorraine'} {b'Ostring', b'Lorraine'}

----- READ -----
Get reandom entry from the set: b'Matte'
Intersection between the two sets: {b'Lorraine'}
Difference between the two sets: {b'Matte', b'Obstberg', b'Breitsch'}
Union of the two sets: {b'Obstberg', b'Breitsch', b'Matte', b'Ostring', b'Lorraine'}

----- UPDATE -----
Move `Matte` from first to second set: {b'Obstberg', b'Breitsch', b'Lorraine'} {b'Matte', b'Ostring', b'Lorraine'}

----- DELETE -----
Remove `Breitsch` from the first set: {b'Obstberg', b'Lorraine'}


## HASH

In [245]:
# CREATE VALUES
print("----- CREATE -----")
r.hmset("buses", {30: "Marzilistrasse", 10: "Ostermundigen", 20: "Wankdorf"})
r.hmset("km", {30: 100, 10: 145.3, 20: 200})
print('Create bus hash:', r.hgetall("buses"))
print('Create KM hash:', r.hgetall("km"))

# READ VALUES FROM DB
print("\n----- READ -----")
print('Get all values of the hash:', r.hvals("buses"))
print('Get all values of the hash:', r.hvals("km"))
print('Get single value by key:', r.hget("buses", 10))
print('Get multiple value by key:', r.hmget("buses", 10, 30))

# UPDATE VALUES IN DB
print("\n----- UPDATE -----")
print('Increment KM of line 30 by 5km to:', r.hincrby("km", 30, 5))
print('Increment KM of line 10 by 5.5km to:', r.hincrbyfloat("km", 10, 5.5))

# DELETE VALUES FROM DB
print("\n----- DELETE -----")
r.hdel("buses", 30)
r.hdel("km", 30)
print('Remove line 30 from bot sets:', r.hgetall("buses"), r.hgetall("km"))

----- CREATE -----
Create bus hash: {b'30': b'Marzilistrasse', b'10': b'Ostermundigen', b'20': b'Wankdorf'}
Create KM hash: {b'30': b'100', b'10': b'145.3', b'20': b'200'}

----- READ -----
Get all values of the hash: [b'Marzilistrasse', b'Ostermundigen', b'Wankdorf']
Get all values of the hash: [b'100', b'145.3', b'200']
Get single value by key: b'Ostermundigen'
Get multiple value by key: [b'Ostermundigen', b'Marzilistrasse']

----- UPDATE -----
Increment KM of line 30 by 5km to: 105
Increment KM of line 10 by 5.5km to: 150.8

----- DELETE -----
Remove line 30 from bot sets: {b'10': b'Ostermundigen', b'20': b'Wankdorf'} {b'10': b'150.8', b'20': b'200'}


## SORTED SET

In [275]:
# For demo puroposes clear the sets every the time we run this cell
r.delete("trams")

# CREATE VALUES
print("----- CREATE -----")
r.zadd("trams", {"Wankdorf": "9", "Weissenbuehl": "3", "Buempliz": "7"})
print('Create a sorted set with these values (ordered by line):', r.zrange("trams", 0, 10))

# READ VALUES FROM DB
print("\n----- READ -----")
print('The set is', r.zcard("trams"), 'items long')
print('There is', r.zcount("trams", 0, 8), 'tram lines with a linenumber lower than 8')
print('The tram going to `Wankdorf` has linenumber', r.zscore("trams", "Wankdorf"))
print('Reqad values in reverse order:', r.zrevrangebyscore("trams", 10, 0))

# UPDATE VALUES IN DB
print("\n----- UPDATE -----")
print('Let us increase the line number of the tram to `Bümpliz` by 1 to:', r.zincrby("trams", 1, "Buempliz"))

# DELETE VALUES FROM DB
print("\n----- DELETE -----")
r.zremrangebyrank("trams", 0, 1)
print('Remove all trams with linenumbers lower than 8:', r.zrange("trams", 0, 10))

----- CREATE -----
Create a sorted set with these values (ordered by line): [b'Weissenbuehl', b'Buempliz', b'Wankdorf']

----- READ -----
The set is 3 items long
There is 2 tram lines with a linenumber lower than 8
The tram going to `Wankdorf` has linenumber 9.0
Reqad values in reverse order: [b'Wankdorf', b'Buempliz', b'Weissenbuehl']

----- UPDATE -----
Let us increase the line number of the tram to `Bümpliz` by 1 to: 8.0

----- DELETE -----
Remove all trams with linenumbers lower than 8: [b'Wankdorf']


## BITMAP
Bitmaps are not an actual data type, but a set of bit-oriented operations defined on the String type.

In [201]:
# CREATE VALUES
r.setbit("bernBit", 5, 1)
r.setbit("bernBit", 3, 1)

# READ VALUES FROM DB
print('If we set the 5th bit of a key we will get:', r.get("bernBit"))
print('The 4th bit is set to:', r.getbit("bernBit", 4))
print('The first bit set to 1 is at position:', r.bitpos("bernBit", 1))
print(r.bitcount("bernBit"), "bits are cureently set to 1")

If we set the 5th bit of a key we will get: b'\x14'
The 4th bit is set to: 0
The first bit set to 1 is at position: 3
2 bits are cureently set to 1


## HYPERLOG LOGS
A HyperLogLog is a probabilistic data structure used in order to count unique things (technically this is referred to estimating the cardinality of a set)

In [203]:
r.pfadd("places", "Bäreplatz", "Eigerplatz", "Helvetiaplatz", "Bäreplatz")
print('Hayperlogs work like sets - so the count of `["Bäreplatz", "Eigerplatz", "Helvetiaplatz", "Bäreplatz"]` is:', r.pfcount("places"))

Hayperlogs work like sets - so the count of `["Bäreplatz", "Eigerplatz", "Helvetiaplatz", "Bäreplatz"]` is: 3
